<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/code/01RAD_Ex04_hw_Rusa_Vojtasek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Team HW 04

## Popis úlohy

V tomto úkolu je cílem provést předzpracování datového souboru, jeho vizualizaci a jednoduchou lineární regresní úlohu, kde se budeme zajímat o ceny nemovitostí. Za tímto účelem využijeme datový set *saratosa_hoouses* z knihovny *moderndive* obsahující výběr 1057 domů.




### Předzpracování dat:




**Data**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.datasets import get_rdataset
from scipy.stats import t,norm

In [ ]:
!pip install rpy2
%load_ext rpy2.ipython
from rpy2.robjects import pandas2ri


In [ ]:
%%R -o saratoga_houses
list_of_packages <- c("tidyverse", "MASS","moderndive")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)
? saratoga_houses

In [ ]:
saratoga_houses.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.pairplot(saratoga_houses)
plt.suptitle('Pair Plot for Saratoga Houses Dataset', y=1.02)
plt.show()

## Otázka 01

Zjistěte, zdali data neobsahují chybějící hodnoty *NA*. Pokud ano, tak rozhodněte zdali můžete příslušná pozorování z dat odstranit a proč. Které proměnné jsou kvantitativní a které kvalitativní? Jeli možno některé zařadit do obou skupin, pro kterou byste se rozhodli? Které proměnné je možné použít jako faktorové ordinální a jaké jako faktorové nominální a proč? Spočtěte základní statistiky pro jednotlivé proměnné.


Odpovědi:
 - Chybí hodnota lot size u **9 pozorování**. Což k celkovému počtu pozorování (1057) je málo a jejich odstranení by nemělo ovlivnit výsledek regrese.
 -
    - **Kvantitativní**: price, living area, bathrooms, bedrooms, fireplaces, lot size, age
    - **kvalitativní**: fireplace. Faktorová nominální proměnná.)
    - **do obou kategorií lze zařadit**: bathrooms, bedrooms, fireplaces. Všechny lze chápat jako faktorové ordinální proměnné.

### Řešení 01:

In [ ]:
missing_values = saratoga_houses.isnull().sum()

rows_with_missing_values = saratoga_houses[saratoga_houses.isnull().any(axis=1)]
print(rows_with_missing_values.shape)
rows_with_missing_values

In [ ]:
saratoga_houses

In [ ]:
saratoga_houses = saratoga_houses.dropna()

In [ ]:
print("Základní statistiky:")
saratoga_houses.describe().round(3)

## Otázka 02

Chceme koupit nemovitost v zahraničí a průzkumem trhu jsme obdřeli předchozí data set *saratoga_houses*. Jelikož ale máme přesnější požadavky a nerozumíme imperiálním jednotkám, potřebujeme data upravit:

* Převeďte cenu nemovitostí z dolarů na koruny v tisících a plochu pozemku a obytnou plochu z akrů a čtverečích stop na $m^2$. (chceck describtion by *? saratoga_houses*)
* Vyberte jen nemovitosti starší 10 let a mladší 50 let, jejichž cena je menší než 7500000 Kč, a plocha pozemku je mezi 500 a 5000 $m^2$.
* Počet koupelen a počet pokojů převeďte na faktorové proměnné o 3 úrovních.

**Dále pracujte jen s takto omezeným datasetem a s proměnnýma *cena*, *plocha_obytna*, *plocha_pozemku* *pocet_pokoju*,*stari_domu*, *pocet_koupelen*, *krb*.**

### Řešení 02:

In [ ]:
saratoga_houses["price"] = ((saratoga_houses["price"] * 23.39) / 1000).round(0)
saratoga_houses["lot_size"] = (saratoga_houses["lot_size"] * 4046.86)
saratoga_houses["living_area"] = (saratoga_houses["living_area"] * 0.0929)

In [ ]:

saratoga_houses = saratoga_houses[(saratoga_houses["age"] >= 10) & (saratoga_houses["age"] <= 50)]


saratoga_houses = saratoga_houses[saratoga_houses["price"] < 7500]


saratoga_houses = saratoga_houses[(saratoga_houses["lot_size"] >= 500) & (saratoga_houses["lot_size"] <= 5000)]


In [ ]:

sns.histplot(saratoga_houses["bedrooms"], discrete=True)
plt.title("bedrooms hist")
plt.show()
sns.histplot(saratoga_houses["bathrooms"])
plt.title("bathrooms hist")
plt.show()

In [ ]:
import pandas as pd

# Recode 'bedrooms' into three categories
bedrooms_bins = [0, 2, 3, 5]


saratoga_houses['bedrooms_cat'] = pd.cut(
    saratoga_houses['bedrooms'].astype(int),
    bins=bedrooms_bins,
    labels=["1-2", "3", "4-5"],
    right=True
)
bathroom_bins = [0, 1, 2.5, 4]


saratoga_houses['bathrooms_cat'] = pd.cut(
    saratoga_houses['bathrooms'],
    bins=[-0.1, 1, 2, 4],  # Adjusting the bins for the specified categories
    labels=['1', '1.5-2', '2.5-3.5'],
    include_lowest=True
)

In [ ]:
sns.histplot(saratoga_houses["bedrooms_cat"], discrete=True)
plt.title("bedrooms hist")
plt.show()
sns.histplot(saratoga_houses["bathrooms_cat"], discrete=True)
plt.title("bathrooms hist")
plt.show()


## Otázka 03

* Porovnejte průměry cen nemovitostí s krbem a bez krbu a otestujte, zdali na hladině významnosti $\alpha = 0.01$ je průměrná cena nemovitostí s krbem větší než průměrná cena nemovitostí bez krbu.

### Řešení 03:


In [ ]:
import statsmodels.stats.weightstats as smw
price_with_fireplace = saratoga_houses[saratoga_houses["fireplace"] == 1]["price"]
price_without_fireplace = saratoga_houses[saratoga_houses["fireplace"] == 0]["price"]

t_test = smw.ttest_ind(price_with_fireplace, price_without_fireplace, alternative="larger")
t_stat, p_value, df = t_test

print(f"t-statistic: {t_stat}, p-value: {p_value}")
print(p_value > 0.001)

# Vizualizace dat

## Otázka 04

* Vykreslete scatterploty pro všechny numerické proměnné, kde bude barevně rozlišeno, zdali se jedná o nemovitost s krbem, nebo bez krbu.
*  Pro proměnné *pocet_pokoju* a *pocet_pater* a *sklep* vykreslete krabicové diagramy (nebo violin ploty), kde odezvou bude *cena*.
* Pro proměnnou *cena* vykreslete histogram spolu s jádrovým odhadem hustoty.


In [ ]:
numeric_columns = ['price', 'living_area', 'fireplace', 'lot_size', 'age']
plt.figure(figsize=(10, 10))

# Create the pairplot
pairplot = sns.pairplot(data=saratoga_houses[numeric_columns],
                         hue='fireplace',
                         palette='Set1',
                         height=4,
                         diag_kind="hist",
                         plot_kws={'s': 30})

# Add a title
pairplot.fig.suptitle("Pairplot of Numerical Variables (Colored by Fireplace)", y=1.02)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Krabicové diagramy
sns.boxplot(data=saratoga_houses, x='bedrooms_cat', y='price', ax=axes[0, 0])
axes[0, 0].set_title('bedrooms vs price')

sns.boxplot(data=saratoga_houses, x='bathrooms_cat', y='price', ax=axes[0, 1])
axes[0, 1].set_title('bathrooms vs price')

sns.boxplot(data=saratoga_houses, x='fireplace', y='price', ax=axes[0, 2])
axes[0, 2].set_title('fireplace vs price')

# Violin ploty
sns.violinplot(data=saratoga_houses, x='bedrooms_cat', y='price', ax=axes[1, 0])
axes[1, 0].set_title('bedrooms vs price')

sns.violinplot(data=saratoga_houses, x='bathrooms_cat', y='price', ax=axes[1, 1])
axes[1, 1].set_title('bathrooms vs price')

sns.violinplot(data=saratoga_houses, x='fireplace', y='price', ax=axes[1, 2])
axes[1, 2].set_title('fireplace vs price')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
sns.histplot(data=saratoga_houses, x='price', kde=True, color="tab:blue")
plt.title('Histogram Cena')
plt.show()

## Otázka 05

Pro kombinace faktorizovaných proměnných *pocet_pokoju*, *pocet_koupelen*  vykreslete cenu nemovitosti, aby bylo na obrázku vidět, jestli se v průměru liší ceny nemovitostí majících více pokojů, nebo více koupelen a zdali jsou zastoupeny všechny kombiance všech úrovních pro dvě zmíněné faktorové proměnné.

In [ ]:
saratoga_houses[["bedrooms_cat", "bathrooms_cat", "price"]].value_counts(ascending=True).reset_index(name='count')

In [ ]:
# Replace 'cena_nemovitosti' with the actual column name for property price
mean_prices = saratoga_houses.groupby(['bedrooms_cat', 'bathrooms_cat'])['price'].mean().reset_index()
mean_prices

In [ ]:
comb_table = saratoga_houses.pivot_table(index='bedrooms_cat', columns='bathrooms_cat', values='price', aggfunc='size', fill_value=0)
sns.heatmap(comb_table, annot=True, cmap='viridis', fmt=".0f")
plt.title('Combination Table')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.pointplot(data=saratoga_houses, x='bedrooms_cat', y='price', hue='bathrooms_cat', dodge=True, markers='o', linestyles='-', palette='viridis')
plt.title('Avg price of houses with respect to the number of bedrooms and bathrooms')
plt.xlabel('bedrooms')
plt.ylabel('Price')
plt.legend(title='bathrooms')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 8))
sns.pointplot(data=saratoga_houses, x='bathrooms_cat', y='price', hue='bedrooms_cat', dodge=True, markers='o', linestyles='-', palette='viridis')
plt.xlabel('bathrooms')
plt.ylabel('price')
plt.legend(title='bedrooms')
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

# Create a grouped bar chart
sns.barplot(
    data=mean_prices,
    x='bedrooms_cat',
    y='price',
    hue='bathrooms_cat',
    palette='viridis'
)

# Set plot labels and title
plt.title('Average Property Price by Number of Bedrooms and Bathrooms')
plt.xlabel('Number of Bedrooms')
plt.ylabel('Average Property Price')
plt.legend(title='Number of Bathrooms', bbox_to_anchor=(1.05, 1), loc='upper left')

# Show the plot
plt.tight_layout()
plt.show()


## Otázka 06

Pro nemovitosti s dvěma ložnicema vykreslete závislost ceny na obytné ploše nemovitosti, kde jednotlivé události označíte barvou podle toho zdali mají krb a velikost bodů v grafu bude odpovídat počtu koupelen (pro tuto úlohu je lepší vzít počet koupelen jako numerickou proměnnou).

**Dále pracujte jen s nemovitostmi se dvěma ložnicemi.**

In [ ]:

saratoga_houses_with_2_bedrooms = saratoga_houses[saratoga_houses["bedrooms"] == 2]
plt.figure(figsize=(10, 6))
sns.scatterplot(data=saratoga_houses_with_2_bedrooms, x="living_area", y="price", hue="fireplace", size="bathrooms", sizes=(150, 20), palette="Set1")
plt.title("Scatterplot of Price vs. Living Area for Houses with Two Bedrooms")
plt.xlabel("Living Area")
plt.ylabel("Price")


In [ ]:
saratoga_houses_with_2_bedrooms.describe()

# Jednoduchý lineární model

## Otázka 07

Sestavte jednoduchý regresní model (s i bez interceptu), kde vysvětlovaná proměnná
bude cena nemovitosti a vysvětlující obytná plocha.   Spočtěte pro oba modely $R^2$ a $F$ statistiky, co nám o modelech říkají. Vyberte jeden z nich a zdůvodněte proč ho preferujete.

Na základě zvoleného modelu odpovězte, zdali cena nemovitosti závisí na obytné ploše  a pokud ano, o kolik se změní očekávaná cena pro nemovitost s obytnou plochou zvětšenou o $20 m^2$?

In [ ]:
Y = saratoga_houses_with_2_bedrooms['price']
X = saratoga_houses_with_2_bedrooms['living_area']

# Add a constant term to include the intercept
X_with_intercept = sm.add_constant(X)

# Fit the model with intercept
model_with_intercept = sm.OLS(Y, X_with_intercept)
results_with_intercept = model_with_intercept.fit()

# Print the summary
print(results_with_intercept.summary())




model_without_intercept = sm.OLS(Y, X)
results_without_intercept = model_without_intercept.fit()

# Print the summary
print(results_without_intercept.summary())

plt.figure(figsize=(10, 6))
plt.scatter(X, Y, alpha=0.5)


plt.plot(
    X,
    results_with_intercept.predict(X_with_intercept),
    color='red',
    label='model with intercept'
)

# Plot the regression line using the fitted model
plt.plot(
    X,
    results_without_intercept.predict(X),
    color='purple',
    label='model without intercept'
)

plt.title('Model without Intercept')
plt.xlabel('Living Area')
plt.ylabel('Price')
plt.legend()
plt.show()


Statistiky $R^2$  a  $F$ pro model s interceptem nám ukazují, že značná část variability v cenách domů je lépe vysvětlena tímto modelem oproti pouhému průměru cen domů.
$R^2$ pro model bez inerceptu nemůžeme porovnávat, jelikož je ovlivěno rozdělením dat. $F$ z definice nemá smysl ji uvažovat pro model bez interceptu.

Vybereme tedy model s interceptem, jelikož nepředpokládáme, že budeme předpovídat cenu pro domy s obytnou plochou blízkou nule.

In [ ]:
slope = results_with_intercept.params['living_area']

price_difference = slope * 20

print(f"Expected change in price for a 20 m^2 increase: {price_difference*1000:.2f} Kč")


## Otázka 08
Sestavte jednoduchý linární model jako v předchozí otázce pro nemovitosti s krbem a bez krbu. Jaký model vykazuje silnější linearní vztah mezi cenou a obytnou plochou? O kolik cena s rostoucí obytnou plochou pro nemovitosti s krbem roste rychleji než pro nemovitosti bez krbu?

Spočtěte 95% konfidenční intervaly pro regresní koeficienty popisující sklon regresní přímky v obou modelech a zjistěte, zdali se protínají. Co z toho můžeme vyvozovat?

Na základě těchto modelů zjistěte o kolik procent bude mít průměrná nemovitost s krbem a obytnou plochou $160m^2$ vyšší očekávanou cenu než průměrná nemovitost o stejné obytné ploše, ale bez krbu.

In [ ]:
houses_with_fireplace = saratoga_houses_with_2_bedrooms[saratoga_houses_with_2_bedrooms["fireplace"]==1]
houses_without_fireplace = saratoga_houses_with_2_bedrooms[saratoga_houses_with_2_bedrooms["fireplace"]==0]

Y_fireplace = houses_with_fireplace['price']
X_fireplace = houses_with_fireplace['living_area']
X_fireplace_with_const= sm.add_constant(X_fireplace)

model_fireplace = sm.OLS(Y_fireplace, X_fireplace_with_const)
results_fireplace = model_fireplace.fit()

print("Model Summary for Properties with a Fireplace:")
print(results_fireplace.summary())

# Model for properties without a fireplace
Y_no_fireplace = houses_without_fireplace['price']
X_no_fireplace = houses_without_fireplace['living_area']
X_no_fireplace_with_const = sm.add_constant(X_no_fireplace)

model_no_fireplace = sm.OLS(Y_no_fireplace, X_no_fireplace_with_const)
results_no_fireplace = model_no_fireplace.fit()
print("\n______________________________________________________________________________\n")
print("Model summary for houses without a fireplace:")
print(results_no_fireplace.summary())




Dle $R^2$ statistiky se zdá, že model pro domy s krbem vykazuje slinější lineární vztah.


In [ ]:
slope_fireplace = results_fireplace.params['living_area']
slope_no_fireplace = results_no_fireplace.params['living_area']
print(f"\nRozdíl zvýšení ceny nemovitostí s krbem  oproti nemovitosti bez krbu je {(results_fireplace.params['living_area']- results_no_fireplace.params['living_area'])*1000:.0f} Kč, pokud se obytná plocha zvýší o jeden m2")


conf_int_fireplace = results_fireplace.conf_int(alpha=0.05) # Can adjust the confidence with alpha
conf_int_fireplace.columns = ['Lower Bound', 'Upper Bound']
print("\nConfidence intervals for parameters model with fireplace")
print(conf_int_fireplace)

conf_int_no_fireplace = results_no_fireplace.conf_int(alpha=0.05) # Can adjust the confidence with alpha
conf_int_no_fireplace.columns = ['Lower Bound', 'Upper Bound']
print("\nConfidence intervals for parameters model without fireplace")
print(conf_int_no_fireplace)

print()
living_area = 160
price_fireplace = results_fireplace.params['const'] + slope_fireplace * living_area
price_no_fireplace = results_no_fireplace.params['const'] + slope_no_fireplace * living_area

price_difference_percentage = ((price_fireplace - price_no_fireplace) / price_no_fireplace) * 100
print(f"Očekávaný rozdíl ceny pro 160 m^2: {price_difference_percentage:.2f}%")


Ano, konfideční intervaly pro sklon regresní přímky se protínají.  Protnutí konfidenčních intervalů naznačuje, že není možné s dostatečnou mírou jistoty tvrdit, že se sklon regresní přímky v obou modelech statisticky významně liší.

## Otázka 9

Vykreslete scatterplot obytné plochy a ceny nemovitostí. Do tohoto grafu vykreslete regresní přímky vybraných modelů pro nemovitosti s krbem a bez něho, jednoltivé body i regresní přímky označte barvou podle toho k jaké skupině přísluší.

Sestrojte 90% konfidenční intervaly okolo očekávaných cen pro jednoltivé skupiny a na jejich základě rozhodněte, zdali a jak se očekávané ceny budou lišit pro nemovitosti s obytnou plochou menší než $120m^2$. Je to porovnávání správné? Zdůvoněte.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Scatter plot for properties with fireplace
plt.figure(figsize=(12, 8))
plt.scatter(X_fireplace, Y_fireplace, color='blue', alpha=0.5, label='With Fireplace')
plt.scatter(X_no_fireplace, Y_no_fireplace, color='green', alpha=0.5, label='Without Fireplace')

# Generate regression lines
X_range = np.linspace(60, 260, 100)  # Adjust range as per your data
X_range_with_const = sm.add_constant(X_range)

# Predicted values and confidence intervals for properties with fireplace
pred_fireplace = results_fireplace.get_prediction(X_range_with_const)
mean_pred_fireplace = pred_fireplace.predicted_mean
conf_int_fireplace_90 = pred_fireplace.conf_int(alpha=0.1)

# Predicted values and confidence intervals for properties without fireplace
pred_no_fireplace = results_no_fireplace.get_prediction(X_range_with_const)
mean_pred_no_fireplace = pred_no_fireplace.predicted_mean
conf_int_no_fireplace_90 = pred_no_fireplace.conf_int(alpha=0.1)

# Plot regression line and confidence interval for properties with fireplace
plt.plot(X_range, mean_pred_fireplace, color='blue', label='Regression Line (With Fireplace)')
plt.fill_between(X_range, conf_int_fireplace_90[:, 0], conf_int_fireplace_90[:, 1], color='blue', alpha=0.2)

# Plot regression line and confidence interval for properties without fireplace
plt.plot(X_range, mean_pred_no_fireplace, color='green', label='Regression Line (Without Fireplace)')
plt.fill_between(X_range, conf_int_no_fireplace_90[:, 0], conf_int_no_fireplace_90[:, 1], color='green', alpha=0.2)

plt.axvline(120, color='k', linestyle='-')
plt.text(x=120, y=-1600, s="120", color='k', size="large", ha='center', va='center')
# Labels and legend
plt.title("Living Area vs. Price with Regression Lines and 90% Confidence Intervals")
plt.xlabel("Living Area")
plt.ylabel("Price")
plt.legend()
plt.show()


## Otázka 10

Vykreslete histogramy pro rezidua modelů z předchozí otázky. Proložte je hustotou normálního rozdělení s nulovou střední hodnotou a rozptylem odpovídajícím $\hat{\sigma}^2$ z jednotlivých modelů.

Co výsledný graf říka o n našich modelech a je toto ověření dostatečné pro validaci model?

Navrněte další úpravy modelu za cílem co nejlépe predikvoat cenu nemovitosti.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

# Residuals for each model
residuals_fireplace = results_fireplace.resid
residuals_no_fireplace = results_no_fireplace.resid

# Plot histograms for residuals with normal density overlay
plt.figure(figsize=(14, 6))

# Histogram and density for properties with a fireplace
plt.subplot(1, 2, 1)
plt.hist(residuals_fireplace, bins=20, edgecolor='k', alpha=0.65, density=True)
plt.title('Histogram of Residuals (With Fireplace)')
plt.xlabel('Residual')
plt.ylabel('Density')

# Overlay normal density with expectation 0
std_fireplace = residuals_fireplace.std()
x_vals = np.linspace(residuals_fireplace.min(), residuals_fireplace.max(), 100)
plt.plot(x_vals, stats.norm.pdf(x_vals, 0, std_fireplace), color='red', label='Normal Density')
plt.legend()

# Histogram and density for properties without a fireplace
plt.subplot(1, 2, 2)
plt.hist(residuals_no_fireplace, bins=20, edgecolor='k', alpha=0.65, density=True)
plt.title('Histogram of Residuals (Without Fireplace)')
plt.xlabel('Residual')
plt.ylabel('Density')

# Overlay normal density with expectation 0
std_no_fireplace = residuals_no_fireplace.std()
x_vals = np.linspace(residuals_no_fireplace.min(), residuals_no_fireplace.max(), 100)
plt.plot(x_vals, stats.norm.pdf(x_vals, 0, std_no_fireplace), color='red', label='Normal Density')
plt.legend()

plt.tight_layout()
plt.show()

# Q-Q plot for each model's residuals
plt.figure(figsize=(14, 6))

# Q-Q plot for properties with fireplace
plt.subplot(1, 2, 1)
stats.probplot(residuals_fireplace, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (With Fireplace)')

# Q-Q plot for properties without fireplace
plt.subplot(1, 2, 2)
stats.probplot(residuals_no_fireplace, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals (Without Fireplace)')

plt.tight_layout()
plt.show()

